# MongoDB – CRUD Operations

As part of this session we will be exploring MongoDB in detail with CRUD operations, Data Structure, Loading MongoDB

* MongoDB CRUD Operations
* Querying nested documents
* Creating denormalized table using python script and creating indexes
* Use of aggregate and project.

## MongoDB CRUD operations
* *show collections*– To list collections(which is similar to tables in RDBMS) -show collections;
* list databases – show databases;
* use retail_db – switch to database retail_db
* NOTE: MongoDB uses JSON documents to store data

*Projection of Data*
* Write a query to get only COMPLETE orders
    * <mark>db.orders.find({"order_status" : "COMPLETE"})</mark>By default 20 are displayed
    * <mark>db.orders.find({"order_status" : "COMPLETE"})</mark>.pretty() is used to beautify.
    * <mark>db.order.find </mark> with enter key gives you the syntax of the function
* Project only order_id, order_date and order_status and not order_customer_id.
* <mark>db.orders.find({"order_status" : "COMPLETE"}, {"order_customer_id" : 0})</mark>(use 1 for displaying output and 0 for not displaying the field)
* Count number of records falling under “COMPLETE” status
* <mark>db.orders.find({"order_status" : "COMPLETE"}).count()</mark>

## Querying nested document
* Look into order_details
    * <mark>db.order_details({"order_id"  :  2})</mark>
* Here we see list of order_items associated with order.We can have denormalized data in mongodb by creating nesting of data like in order_items thus removing redundancy.
* How to query from nested document?
    * In order_items check only those records where order_quantity is greater than 1

In [ ]:
db.order_details.
  aggregate(
    [{$unwind : '$order_items'}, 
     {$match : {
      'order_id' : 2, 
      'order_items.order_item_quantity' : 5
     }}
    ])    

**Use of unwind**  – for deconstructing an array field from the input documents to output a document for each element and get the data in a way as in RDBMS.This function comes as part of aggregate.You can also apply filtering on top of it using match.

* In order_items check only those records where order_quantity is less than 5

In [ ]:
db.order_details.
  aggregate(
    [{$unwind : '$order_items'},
     {$match  : {
       'order_id' : 2,
       $lt  : 'order_items.order_item_quantity' : {$lt  : 5}
       }}
    ])

*Connecting to MongoBooster*
Go to connect -> ssh -> check use ssh tunnel -> ip add gw02.itversity.com -> username :training

Go to basic -> gw01.itversity.com  -> save and connect

*Dropping  the table and indexes*
* <mark>db.order_details.drop()</mark>
* <mark>db.order_items.getIndexes()</mark>
* <mark>db.order_items.dropIndex(“order_item_order_id_1”)</mark>
* If in output ok is 1 then it is successfull

## Create denormalized table
* First we write the python script as Python can seemlessly work with Python. Python dict is more closer to json. The script creates denormalised table order_details, for each and every orders if there are corresponding child elements in order_items, I want to create collections under each order and insert in order_details.
* Run the script in background
* <mark>nohup python orders_load.py &</mark>
* Connect to mongoDB and use retail_db
    * <mark>db.order_details.find({}).count()</mark>
* We need to create index to improve performance
    * <mark>db.order_items.createIndex({"order_item_order_id" : 1})</mark>
    * <mark>db.order_items.getIndexes()</mark>

## Using aggregate and project

In [ ]:
//mongodb-queries-01-count-by-order-status.json
// Count by order_status
db.orders.
  aggregate([{"$group" : 
                { "_id" : "$order_status", "status_count" : {"$sum" :1 }}
             }, 
             {"$project" : 
               { "order_status" : "$_id" , "status_count" : 1, "_id" : 0 }
             }])

In [ ]:
//mongodb-queries-02-revenue-for-eachorder-status.json
// Get revenue for each order
db.order_items.find({"order_item_order_id" : 2})   

db.order_items.
  aggregate([{"$group" : 
               { "_id" : "$order_item_order_id", "order_revenue" :{ "$sum" : "$order_item_subtotal" }}
             },
             {"$project" : 
               { "order_item_order_id" : "$_id", "order_revenue" : 1, "_id" : 0 }
             }])

In [ ]:
//mongodb-queries-03-filter-for-order-id.json
// Get revenue for order_id 2
db.order_items.
  aggregate([{ "$match" : {"order_item_order_id" : 2}},
             {"$group" : 
               { "_id" : "$order_item_order_id", "order_revenue" : {"$sum" : "$order_item_subtotal"}}
             },
             {"$project" : { "order_item_order_id": "$_id", "order_revenue" : 1, "_id" : 0 }
             }])

*Refer documentation*

https://docs.mongodb.com/manual/crud/